## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Kavieng,PG,-2.5744,150.7967,82.38,71,98,8.08,['overcast clouds']
1,1,Shellbrook,CA,53.2168,-106.4011,64.80,45,8,3.56,['clear sky']
2,2,Roald,NO,62.5833,6.1333,59.32,94,75,2.30,['broken clouds']
3,3,Thompson,CA,55.7435,-97.8558,64.56,36,75,8.05,['broken clouds']
4,4,Aasiaat,GL,68.7098,-52.8699,41.38,69,31,14.61,['scattered clouds']


In [26]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you desire for your trip?  "))
max_temp = float(input("What is the maximum temperature you desire for your trip?  "))

In [27]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & 
                                       (city_data_df["Max Temp"] >= min_temp)] 

preferred_cities_df.head(10)
len(preferred_cities_df)

80

In [28]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.isnull().sum()

City_ID                0
City                   0
Country                0
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [29]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows. (just in case a random data set pulls some with nas)
clean_df = preferred_cities_df.dropna()

In [30]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
4,Aasiaat,GL,41.38,['scattered clouds'],68.7098,-52.8699,
11,Pangnirtung,CA,47.80,['overcast clouds'],66.1451,-65.7125,
34,Chokurdakh,RU,46.72,['scattered clouds'],70.6333,147.9167,
40,New Norfolk,AU,46.92,['overcast clouds'],-42.7826,147.0587,
43,Bluff,NZ,46.80,['overcast clouds'],-46.6000,168.3333,
46,Ushuaia,AR,37.06,['broken clouds'],-54.8000,-68.3000,
49,Qaanaaq,GL,46.67,['broken clouds'],77.4840,-69.3632,
50,Punta Arenas,CL,42.91,['light rain'],-53.1500,-70.9167,
54,Mataura,NZ,46.47,['overcast clouds'],-46.1927,168.8643,
61,Port Augusta,AU,47.93,['overcast clouds'],-32.5000,137.7667,


In [31]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000, # this is defined in meters by default
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Vacation Error 404: Hotel not found!")
        

Vacation Error 404: Hotel not found!
Vacation Error 404: Hotel not found!
Vacation Error 404: Hotel not found!
Vacation Error 404: Hotel not found!
Vacation Error 404: Hotel not found!
Vacation Error 404: Hotel not found!
Vacation Error 404: Hotel not found!
Vacation Error 404: Hotel not found!
Vacation Error 404: Hotel not found!
Vacation Error 404: Hotel not found!
Vacation Error 404: Hotel not found!
Vacation Error 404: Hotel not found!
Vacation Error 404: Hotel not found!
Vacation Error 404: Hotel not found!
Vacation Error 404: Hotel not found!
Vacation Error 404: Hotel not found!
Vacation Error 404: Hotel not found!
Vacation Error 404: Hotel not found!
Vacation Error 404: Hotel not found!
Vacation Error 404: Hotel not found!


In [35]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df[hotel_df["Hotel Name"] != ""]
clean_hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
4,Aasiaat,GL,41.38,['scattered clouds'],68.7098,-52.8699,Hotel Aasiaat Sømandshjem
11,Pangnirtung,CA,47.80,['overcast clouds'],66.1451,-65.7125,Kilabuk Lodge
40,New Norfolk,AU,46.92,['overcast clouds'],-42.7826,147.0587,The Shingles Riverside Cottages
43,Bluff,NZ,46.80,['overcast clouds'],-46.6000,168.3333,Bluff Homestead - Guesthouse & Campervan Park
46,Ushuaia,AR,37.06,['broken clouds'],-54.8000,-68.3000,Albatross Hotel
49,Qaanaaq,GL,46.67,['broken clouds'],77.4840,-69.3632,Qaanaaq Hotel
50,Punta Arenas,CL,42.91,['light rain'],-53.1500,-70.9167,Hotel Hain
54,Mataura,NZ,46.47,['overcast clouds'],-46.1927,168.8643,Ellie's Villa
61,Port Augusta,AU,47.93,['overcast clouds'],-32.5000,137.7667,Majestic Oasis Apartments
69,Tasiilaq,GL,44.76,['broken clouds'],65.6145,-37.6368,The Red House


In [36]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"

# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")
len(clean_hotel_df)

60

In [37]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Weather Description</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]


In [38]:
# 11a. Add a marker layer for each city to the map. 
max_temp = clean_hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp, dissipating=False,
                                 max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
# 11b. Display the figure

fig


Figure(layout=FigureLayout(height='420px'))